In [6]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD
# !pip install plotly
# import plotly.plotly as py
# from plotly.graph_objs import *
# import plotly.tools as tls
# from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# import additional packages to insepct data and clean them
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [23]:
# import helping functions from the implementation file
#from proj1_helpers import load_csv_data
from proj1_helpers import *
import implementations
from additional_implementations import *
import seaborn as sns

In [9]:
# import zipped files from the github repository
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

In [10]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

In [11]:
df_train_gau=df_train.drop(columns=['Prediction', 'Id'])

In [12]:
df_train_gau[df_train_gau==-999.0]=0

In [13]:
y_train_gau=np.array(df_train['Prediction'])
y_train_gau[np.where(y_train_gau=='b')] = -1.
y_train_gau[np.where(y_train_gau=='s')] = 1.
y_train_gau=y_train_gau.astype('float')

In [14]:
tx_train_gau_polynomial=polynomial_features_simple(df_train_gau, 9)

In [15]:
# lambdas=np.logspace(-10,-2,15)
# mses=[]
# ws=[]
# for lambda_ in lambdas:
#     w,loss=implementations.ridge_regression(y_train_gau,tx_train_gau_polynomial,lambda_)
#     mses.append(loss)
#     ws.append(w)
# #cross_validation_visualization(lambdas, mses, mses)
# w_final=ws[np.argmin(mses)]
# mses[np.argmin(mses)]

In [19]:
lambdas=np.logspace(-10,-2,10)
degrees=np.linspace(1,15,3).astype('int')
mses= np.zeros((len(degrees), len(lambdas)))

for ind_degree,degree in enumerate(degrees):
    tx_train_gau_polynomial=polynomial_features_simple(df_train_gau, degree)
    for ind_lambda,lambda_ in enumerate(lambdas):
        w,loss=implementations.ridge_regression(y_train_gau,tx_train_gau_polynomial,lambda_)
        mses[ind_degree, ind_lambda]=loss
        
mse_final, degree_final,lambda_final=get_best_parameters(degrees, lambdas, mses)
tx_train_gau_polynomial=polynomial_features_simple(df_train_gau, degree_final)
w_final,loss=implementations.ridge_regression(y_train_gau,tx_train_gau_polynomial,lambda_final)

print('MSE: ',mse_final,'   Degree: ',degree_final,'   Lambda: ',lambda_final)
print('')
print('Ws: ',w_final)

MSE:  28.156430834296494    Degree:  8    Lambda:  5.994842503189421e-09

Ws:  [-1.35920207e+01 -4.90403746e-01  1.14650361e-02 -9.31723762e-05
  3.62961474e-07 -7.58490040e-10  8.68839510e-13 -5.12717919e-16
  1.21559587e-19  1.31597429e-01 -6.26190953e-03  7.43072690e-05
 -3.86110854e-07  1.01338436e-09 -1.36008470e-12  8.39465433e-16
 -1.60336417e-19 -1.63544453e-01  1.27570711e-03 -8.54705620e-06
  3.49835260e-08 -7.76325638e-11  9.14874726e-14 -5.37884823e-17
  1.23826513e-20 -5.16785036e-02  9.87617970e-04 -9.96743402e-06
  5.41953229e-08 -1.60307244e-10  2.54672759e-13 -2.01307861e-16
  6.09472001e-20 -1.34034340e+00  2.71302283e+00 -4.66745911e+00
  3.39243895e+00 -1.17857775e+00  2.11918752e-01 -1.91389813e-02
  6.88898165e-04 -4.99571828e-02  1.20593124e-04 -1.52627910e-07
  1.09993003e-10 -4.67345319e-14  1.15239443e-17 -1.51782072e-21
  8.22357184e-26 -5.89116574e-01 -3.02914817e-02  2.80741280e-03
  2.40657183e-04 -1.98647295e-05 -1.05196771e-08  9.25132678e-08
 -5.2784578

In [25]:
y_pred=predict_labels(w_final,tx_train_gau_polynomial)
list(y_pred*y_train_gau).count(1.)/len(tx_train_gau_polynomial)


0.81104

In [ ]:
df_test_gau=df_test

In [ ]:
df_test_gau=df_test_gau.drop(columns=['Id', 'Prediction'])

In [ ]:
df_test_gau[df_test_gau==-999.0]=0

In [ ]:
tx_test_gau_polynomial=polynomial_features_simple(df_test_gau,9)

In [ ]:
# tx_test=tx_test_gau_polynomial
# y_pred=predict_labels(w_final,tx_test)
# create_csv_submission(df_test['Id'], y_pred, 'trial_ridge_polynomial_gau_basic')

In [ ]:
gau_std=pd.read_csv('trial_ridge_polynomial_gau')
gau_basic=pd.read_csv('trial_ridge_polynomial_gau_basic')

check=gau_basic['Prediction']*gau_std['Prediction']
check.value_counts().plot(kind='bar')